<a href="https://colab.research.google.com/github/CLAIR-LAB-TECHNION/ICAPS-24/blob/main/notebooks/ICAPS_SummerSchool_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why we need TAMP - running example



*  n-table blocks world

* challenges:
  * continuous state space (large search space)
  * multiple way to solve the task
  * ...


version 1:  per block
vesion 2 : high level request (2 yellow blocks on table 2)
version 3: accountig for movement costs

# Task Planning with AIDM


* pddl version of NTBW
  * brief intro to pddl
* solving NTBW with best first search
  * visualize symbolic 2D (maybe)




# Why we need TAMP - revisited



*   What's missing in task planning and why we need motion planning
*   Why seperate task planning and motion planning is not enough
* do task plan with motion planning (random sample) in mujoco


# Mujoco - (very) brief introduction (more on this tomorrow)



* Mujoco as an example physics engine
* demonstrate inverse kinematics through challenges
  * e.g., give actions to put arm in certain config
